Read the data in from API and local file. Assumes notebook is running in the same directory where the 'Fire Data' directory is located. 

In [7]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("tm6d-hbzd", limit = 250000)#limit=300000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

#Get the firehouse location information
firehouse_info = client.get("hc8x-tcnd", limit=10000)

firehouse_df = pd.DataFrame.from_records(firehouse_info)

#Get the firebox location information
firebox_locs = pd.read_csv("Fire Data\\Fire Boxes.csv", names=['LAT','LONG','fire_box','nearest_intersection'])

In [8]:
#convert dates to date time objects
results_df['arrival_date_time']=pd.to_datetime(results_df['arrival_date_time'])
results_df['incident_date_time']=pd.to_datetime(results_df['incident_date_time'])
results_df['last_unit_cleared_date_time']=pd.to_datetime(results_df['last_unit_cleared_date_time'])

In [9]:
#Limit results to 2018
results_df = results_df[results_df['incident_date_time'] > pd.to_datetime('2018-01-01 00:00:00')]

In [10]:
#Convert the columns to appropriate types. Probably more to do here. 
results_df['response_time'] = (results_df['arrival_date_time'] - results_df['incident_date_time'])
results_df['response_time'] = pd.to_numeric(results_df['response_time'])
results_df['units_onscene'] = pd.to_numeric(results_df['units_onscene'])
results_df['total_incident_duration'] = pd.to_numeric(results_df['total_incident_duration'])
results_df['story_fire_origin_count'] = pd.to_numeric(results_df['story_fire_origin_count'])
results_df['response_time'] = results_df['response_time']/ 60000000000

results_df['fire_box'] = results_df['fire_box'].astype('str')

In [11]:
#Just to show you how we are converting the firebox to match the fire_box dataframe for merging
results_df.borough_desc.unique()

array(['2 - Bronx', '4 - Brooklyn', '1 - Manhattan', '5 - Queens',
       '3 - Staten Island'], dtype=object)

In [12]:
#Add the borough code to the firebox column
results_df.loc[results_df['borough_desc'].str.match('1 - Manhattan'), 'fire_box'] = 'M' + results_df.fire_box
results_df.loc[results_df['borough_desc'].str.match('2 - Bronx'), 'fire_box'] = 'X' + results_df.fire_box
results_df.loc[results_df['borough_desc'].str.match('3 - Staten Island'), 'fire_box'] = 'R' + results_df.fire_box
results_df.loc[results_df['borough_desc'].str.match('4 - Brooklyn'), 'fire_box'] = 'B' + results_df.fire_box
results_df.loc[results_df['borough_desc'].str.match('5 - Queens'), 'fire_box'] = 'Q' + results_df.fire_box

In [13]:
#Merge the fire_box dataframe with the overall dataframe
final_df = pd.merge(results_df, firebox_locs, on = 'fire_box')

Display the final dataframe. 

In [14]:
final_df.head(100)

,action_taken1_desc,action_taken2_desc,action_taken3_desc,aes_presence_desc,arrival_date_time,borough_desc,co_detector_present_desc,detector_presence_desc,fire_box,fire_origin_below_grade_flag,...,standpipe_sys_present_flag,story_fire_origin_count,street_highway,total_incident_duration,units_onscene,zip_code,response_time,LAT,LONG,nearest_intersection
0,"42 - HazMat detection, monitoring, sampling, &...",51 - Ventilate,86 - Investigate,NaN,2018-07-01 00:00:32,2 - Bronx,Yes,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1648.0,2.0,10453,5.050000e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
1,86 - Investigate,NaN,NaN,NaN,2018-06-02 07:48:48,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1671.0,6.0,10453,5.133333e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
2,"00 - Action taken, other",NaN,NaN,NaN,2018-05-17 14:07:16,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,2820.0,1.0,10453,4.583333e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
3,86 - Investigate,NaN,NaN,NaN,2018-05-07 06:19:36,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1235.0,1.0,10453,6.166667e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
4,"00 - Action taken, other",NaN,NaN,NaN,2018-05-02 19:47:00,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,2485.0,1.0,10453,1.061667e+01,-73.922135,40.849047,Undercliff Ave & W 175th St
5,"00 - Action taken, other",NaN,NaN,NaN,2018-04-24 00:03:08,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1434.0,1.0,10453,6.433333e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
6,"00 - Action taken, other",NaN,NaN,NaN,2018-04-18 09:50:55,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1165.0,1.0,10453,9.366667e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
7,86 - Investigate,111 - Hooked up to hydrant,215 - Check for extension,NaN,2018-04-07 16:23:32,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,1301.0,4.0,10453,3.633333e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
8,"00 - Action taken, other",NaN,NaN,NaN,2018-04-02 05:01:44,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,3072.0,1.0,10453,5.716667e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
9,11 - Extinguishment by fire service personnel,51 - Ventilate,NaN,NaN,2018-03-23 19:49:07,2 - Bronx,NaN,NaN,X2928,NaN,...,NaN,NaN,UNDERCLIFF AVE,2147.0,5.0,10453,6.333333e+00,-73.922135,40.849047,Undercliff Ave & W 175th St
